# 04_07: SOLUTION - Evaluating your GraphRAG Pipeline

In [ ]:
import os
from dotenv import load_dotenv

from langchain.evaluation.qa.eval_chain import QAEvalChain
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

In [ ]:
load_dotenv(dotenv_path='../../.env')

URI = os.getenv("NEO4J_URI")
USER = os.getenv("NEO4J_USER")
PWD = os.getenv("NEO4J_PWD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
enhanced_graph = Neo4jGraph(url=URI, username=USER, password=PWD, enhanced_schema=True)
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4o", temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [ ]:
examples = [
    {"query": "What sports is the International Ski And Snowboard Federation responsible for?",
     "answer": "Alpine Skiing, Freestyle Skiing, Snowboarding, Nordic Combined, Ski Jumping, Cross-Country Skiing"},
    {"query": "What activity are ski poles not used in?", 
     "answer": "Ski jumping"},
    {"query": "Who do athletes get help from?",
     "answer": "Coaches, Peer Mentors, and Sports Psychologists"},
]

In [ ]:
def evaluate_graph_rag(graph_chain, examples):

    # Generate predictions by querying the graph
    predictions = []
    for ex in examples:
        graph_response = graph_chain.invoke({"query": ex["query"]})
        predictions.append({"result": graph_response["result"].strip()})

    # Run evaluation
    eval_chain = QAEvalChain.from_llm(llm)
    results = eval_chain.evaluate(examples, predictions)

    # Print output
    correct = 0
    for i, res in enumerate(results):
        print(f"Query: {examples[i]['query']}")
        print(f"Prediction from graph: {predictions[i]['result']}")
        print(f"Gold answer: {examples[i]['answer']}")
        print(f"Grade: {res['results']}")
        print("---")
        if res["results"] == "CORRECT":
            correct += 1

    accuracy = correct / len(examples)
    print(f"Graph QA Accuracy: {accuracy:.2f}")

In [ ]:
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to
answer questions about skiing.
Convert the user's question based on the schema.
When you are presented with query properties such as id's like "alpine touring",
be sure to convert the first letter to capital case, such as "Alpine Touring"
before you run the Cypher query.

NEVER USE THE MENTIONS RELATIONSHIP FOR ANY ANSWERS!

For example, if I were to ask "Tell me about telemark skiing," you should create
a Cypher query that finds all nodes with the id "Telemark Skiing" and then find
all nodes connected to those nodes and use those to forumulate your answer, like this:

MATCH (a {{id: "Telemark Skiing"}})-[r]-(b)
RETURN a, r, b

You MUST use the question to infer the node and relationship labels.
"What sports is the International Ski and Snowboard Federation responsible for?"
I would create a query like:

MATCH (a)-[r:RESPONSIBLE_FOR]-(b:Sport)
RETURN a, r, b

Schema: {schema}
Question: {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

graph_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=enhanced_graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    allow_dangerous_requests=True
)

In [ ]:
evaluate_graph_rag(graph_chain, examples)

In [ ]:
examples = [
    {"query": "What training method does the International Ski and Snowboard Federation use?",
     "answer": "Grass skiing"},
    {"query": "How does Research play a role?",
     "answer": "Research improves competition by looking at Surface Science and Friction Reduction and has implications for Competitive Performance and Recreational Enjoyment"},
    {"query": "What country was the men's overall winner of the 2024–25 Fis Alpine Ski World Cup representing?",
     "answer": "Switzerland"},
]

In [ ]:
CYPHER_GENERATION_TEMPLATE = """

You are an expert Neo4j developer. Your job is to generate a Cypher query that 
will traverse exactly 1–3 hops out from your focus node.

Steps:

1. Identify the main entity in the question and Capitalize Multi-Word IDs
2. Write a variable-length MATCH using the new syntax:
   MATCH (a {{id: "{{entity}}"}})-[r]-{{1,3}}(b)
   RETURN a, r, b;

You MUST use {{1,3}} for setting the number of hops instead of [r*0..3].
Do NOT use WHERE length(r) >= 1 AND length(r) <= 3 to specify it either.

For example, if you were asked a question like "What equipment is used in
the Paralympic Nordic Skiing?" you would construct this Cypher query:

MATCH (a {{id: "Paralympic Nordic Skiing"}})-[r]-{1,3}(b:Equipment)
RETURN a, r, b

Schema:
{schema}

Question:
{question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

enhanced_graph_chain = GraphCypherQAChain.from_llm(
    graph=enhanced_graph,
    llm=llm,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    allow_dangerous_requests=True,
)

In [ ]:
evaluate_graph_rag(graph_chain, examples)